# Trabajo 1 - Nicolas Maldonado - AX840282
## Preparacion de Datos
8 / Diciembre / 2023<br>
Master en Ciencia de Datos y Aprendizaje Automatico

### Descripción: Aprobación de crédito.

### Objetivo:
Aprobación de solicitudes de tarjetas de crédito. Todos los nombres y valores de los atributos se han cambiado a símbolos sin sentido para proteger la confidencialidad de los datos.
#### Objetuivo adicional: Superar el accuracy score de 87.9%

Empezamos cargando las librerias y el dataset, para este caso, si bien son los datos del dataset ubicado en https://www.kaggle.com/datasets/impapan/credit-approval-data-set.

In [17]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import sklearn

import pandas as pd
import numpy as np

In [8]:
df =  pd.read_csv('credit_approval.csv')
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,class
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


Empezamos inspeccionando los links de los modelos propuestos sobre Kaggle para el mismo dataset: <br>
https://www.kaggle.com/code/michaelcai2021/predicting-credit-card-approvals-with-sklearn <br>
https://www.kaggle.com/code/dilipkumarb/credit-card-approval-classification-model

De ellos podemos extraer la siguiente información, para nombrar las columnas y darles un poco de significado, esto debido a que las columnas y los datos fueron anonimizados para proteger la información confidencial de los usuarios:


    Inspecting the data
    After the initial inspection, we can see all attribute names and values have been changed to meaningless symbols to protect the confidentiality of the data. The following data description is from original dataset website. Let's inspect each column

    The probable features in a typical credit card application are:

    Attribute Information:

    A1: b, a. Gender
    A2: continuous. Age
    A3: continuous. Debt
    A4: u, y, l, t. Married
    A5: g, p, gg. BankCustomer
    A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff. EducationLevel
    A7: v, h, bb, j, n, z, dd, ff, o. Ethnicity
    A8: continuous. YearsEmployed
    A9: t, f. PriorDefault
    A10: t, f. Employed
    A11: continuous. CreditScore
    A12: t, f. DriversLicense
    A13: g, p, s. Citizen
    A14: continuous. ZipCode
    A15: continuous. Income
    A16: +,- (class attribute) ApprovalStatus


In [35]:
columnas=['Gender', 'Age', 'Debt', 'Married', 'BankCustomer', 'EducationLevel', 'Ethnicity', 'YearsEmployed', 'PriorDefault', 'Employed', 'CreditScore', 'DriversLicense', 'Citizen', 'ZipCode', 'Income','ApprovalStatus']
df.columns = columnas
df

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


Empezamos realizando una exploración inicial de los datos con un info y un describe, para ver nulos y en datos numericos ver promedios, maximos, minimos, etc.

In [10]:
df.describe()

,Debt,YearsEmployed,CreditScore,Income
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          690 non-null    object 
 1   Age             690 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         690 non-null    object 
 4   BankCustomer    690 non-null    object 
 5   EducationLevel  690 non-null    object 
 6   Ethnicity       690 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  ZipCode         690 non-null    object 
 14  Income          690 non-null    int64  
 15  ApprovalStatus  690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


Empezamos a revisar primero la seleccion de caracteristicas, para ello vamos a tratar las variables categoricas. Revisamos las caracteristicas categoricas para validar los valores que traen. 

In [14]:
for c in df:
    lista = df[c].unique()
    if lista.size < 20:
        print(f"{c}:", lista)

Gender: ['b' 'a' '?']
Married: ['u' 'y' '?' 'l']
BankCustomer: ['g' 'p' '?' 'gg']
EducationLevel: ['w' 'q' 'm' 'r' 'cc' 'k' 'c' 'd' 'x' 'i' 'e' 'aa' 'ff' 'j' '?']
Ethnicity: ['v' 'h' 'bb' 'ff' 'j' 'z' '?' 'o' 'dd' 'n']
PriorDefault: ['t' 'f']
Employed: ['t' 'f']
DriversLicense: ['f' 't']
Citizen: ['g' 's' 'p']
ApprovalStatus: ['+' '-']


Vemos como el dataset cuenta con varios datos en '?', empezamos convirtiendolos en NaN para trabajar mejor procesando y volvemos a hacer un info.

In [16]:
for c in df:
    if df[c].dtype == 'object':
        df[c] = df[c].replace('?',np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          678 non-null    object 
 1   Age             678 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         684 non-null    object 
 4   BankCustomer    684 non-null    object 
 5   EducationLevel  681 non-null    object 
 6   Ethnicity       681 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  ZipCode         677 non-null    object 
 14  Income          690 non-null    int64  
 15  ApprovalStatus  690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


Ahora si empezamos a preprocesar la data, si bien queremos evitar al maximo el data leakage, vamos a revisar cantidad de categorias, maximos y minimos de la totalidad de los datos para tener en cuenta todos los valores posibles. Empezamos tratando nulos.

In [18]:
df.isnull().any(axis=1).sum()

37

In [22]:
for i in df:
    # count number of rows with missing values
    n_miss = df[i].isna().sum()
    perc = n_miss / df.shape[0] * 100
    print('> %s, Missing: %d (%.1f%%)' % (i, n_miss, perc))

> Gender, Missing: 12 (1.7%)
> Age, Missing: 12 (1.7%)
> Debt, Missing: 0 (0.0%)
> Married, Missing: 6 (0.9%)
> BankCustomer, Missing: 6 (0.9%)
> EducationLevel, Missing: 9 (1.3%)
> Ethnicity, Missing: 9 (1.3%)
> YearsEmployed, Missing: 0 (0.0%)
> PriorDefault, Missing: 0 (0.0%)
> Employed, Missing: 0 (0.0%)
> CreditScore, Missing: 0 (0.0%)
> DriversLicense, Missing: 0 (0.0%)
> Citizen, Missing: 0 (0.0%)
> ZipCode, Missing: 13 (1.9%)
> Income, Missing: 0 (0.0%)
> ApprovalStatus, Missing: 0 (0.0%)


In [24]:
df_nulos = df[df.isnull().any(axis=1)]
df_nulos

,Gender,Age,Debt,Married,BankCustomer,EducationLevel,Ethnicity,YearsEmployed,PriorDefault,Employed,CreditScore,DriversLicense,Citizen,ZipCode,Income,ApprovalStatus
71,b,34.83,4.000,u,g,d,bb,12.500,t,f,0,t,g,NaN,0,-
83,a,NaN,3.500,u,g,d,v,3.000,t,f,0,t,g,00300,0,-
86,b,NaN,0.375,u,g,d,v,0.875,t,f,0,t,s,00928,0,-
92,b,NaN,5.000,y,p,aa,v,8.500,t,f,0,f,g,00000,0,-
97,b,NaN,0.500,u,g,c,bb,0.835,t,f,0,t,s,00320,0,-
202,b,24.83,2.750,u,g,c,v,2.250,t,t,6,f,g,NaN,600,+
206,a,71.58,0.000,NaN,NaN,NaN,NaN,0.000,f,f,0,f,p,NaN,0,+
243,a,18.75,7.500,u,g,q,v,2.710,t,t,5,f,g,NaN,26726,+
248,NaN,24.50,12.750,u,g,c,bb,4.750,t,t,2,f,g,00073,444,+
254,b,NaN,0.625,u,g,k,v,0.250,f,f,0,f,g,00380,2010,-


Considerando lo anterior, vamos a realizar una primera prueba eliminando todos aquellos registros con datos nulos y hacemos una prueba inicial para obtener un Score desde el cual vamos a partir a mejorar. Observamos tambien que el tarjet se encuentra categorico con "+" y "-", razon por la cual utilizamos el LabelEncoder y posteriormente tomamos todas aquellas columnas numericas.

In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder

df_clean = df.dropna(inplace=False)
label_encoder = LabelEncoder()  # OJO! LabelEncoder para la target
df_clean["ApprovalStatus"] = label_encoder.fit_transform(df_clean["ApprovalStatus"])
data = df_clean.select_dtypes(include=['float64','int64']).values

X, y = data[:, :-1], data[:, -1]
# Definimos el modelo inicial, probaremos con un random forest y empezamos haciendo uso de la libreria Pipeline
model = RandomForestClassifier()
pipeline = Pipeline(steps=[('m', model)])

# Definimos la evaluación del modelo y lo aplicamos
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=96)
scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Mean Accuracy eliminando los nulos: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean Accuracy eliminando los nulos: 0.297 (0.036)


Empezamos a codificar las variables categoricas. Para ello nuevamente comparamos la cantidad de categorias por cada columna.

In [110]:
for c in df:
    lista = df[c].unique()
    if lista.size < 20:
        print(f"{c}:", lista)
print(df.info())

Gender: ['b' 'a' nan]
Married: ['u' 'y' nan 'l']
BankCustomer: ['g' 'p' nan 'gg']
EducationLevel: ['w' 'q' 'm' 'r' 'cc' 'k' 'c' 'd' 'x' 'i' 'e' 'aa' 'ff' 'j' nan]
Ethnicity: ['v' 'h' 'bb' 'ff' 'j' 'z' nan 'o' 'dd' 'n']
PriorDefault: ['t' 'f']
Employed: ['t' 'f']
DriversLicense: ['f' 't']
Citizen: ['g' 's' 'p']
ApprovalStatus: ['+' '-']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          678 non-null    object 
 1   Age             678 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         684 non-null    object 
 4   BankCustomer    684 non-null    object 
 5   EducationLevel  681 non-null    object 
 6   Ethnicity       681 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore 

De lo anterior notamos que tenemos nulos en varios campos, tambien encontramos que Age fue cargado como object y no como float, asi que procedemos a castear la columna a flotante. Adicionalmente eliminamos la fila de codigo postal, campo que podriamos suponer no es relevante en cuanto a la adjudicacion de prestamos. Y separamos como tal los tipos de columnas y los numericos.

In [133]:
df_trated = df.copy()
y = df_trated['ApprovalStatus']
df_trated = df_trated.astype({'Age': 'float64'})
df_trated.drop(['ZipCode','ApprovalStatus'], axis=1, inplace=True)
categorical_ix = df_trated.select_dtypes(include='object').columns 
numerical_ix = df_trated.select_dtypes(include=['float64','int']).columns

Ya sin la columna de codigo postal que tenia algunos nulos, volvermos a realizar un info para visualizar cantidad de nulos que quedan y sobre que tipo de columans se tienen.

In [134]:
df_trated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          678 non-null    object 
 1   Age             678 non-null    float64
 2   Debt            690 non-null    float64
 3   Married         684 non-null    object 
 4   BankCustomer    684 non-null    object 
 5   EducationLevel  681 non-null    object 
 6   Ethnicity       681 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  Income          690 non-null    int64  
dtypes: float64(3), int64(2), object(9)
memory usage: 75.6+ KB


Visualizamos que unicamente tenemos nulos sobre las variables categoricas y la caracteristica edad, esta ultima la imputaremos, pero para los demas casos, probaremos primero aliminando estas instancias completas.

In [130]:
df_trated = df_trated.dropna(subset = categorical_ix, inplace=False)
df_trated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 671 entries, 0 to 689
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          671 non-null    object 
 1   Age             659 non-null    float64
 2   Debt            671 non-null    float64
 3   Married         671 non-null    object 
 4   BankCustomer    671 non-null    object 
 5   EducationLevel  671 non-null    object 
 6   Ethnicity       671 non-null    object 
 7   YearsEmployed   671 non-null    float64
 8   PriorDefault    671 non-null    object 
 9   Employed        671 non-null    object 
 10  CreditScore     671 non-null    int64  
 11  DriversLicense  671 non-null    object 
 12  Citizen         671 non-null    object 
 13  Income          671 non-null    int64  
dtypes: float64(3), int64(2), object(9)
memory usage: 78.6+ KB


Apartir de este momento comenzamos a manipular las columnas de acuerdo a si son categoricas o numericas

In [132]:
for c in categorical_ix:
    lista = df_trated[c].unique()
    if lista.size < 20:
        print(f"{c}:", lista)

Gender: ['b' 'a']
Married: ['u' 'y' 'l']
BankCustomer: ['g' 'p' 'gg']
EducationLevel: ['w' 'q' 'm' 'r' 'cc' 'k' 'c' 'd' 'x' 'i' 'e' 'aa' 'ff' 'j']
Ethnicity: ['v' 'h' 'bb' 'ff' 'j' 'z' 'o' 'dd' 'n']
PriorDefault: ['t' 'f']
Employed: ['t' 'f']
DriversLicense: ['f' 't']
Citizen: ['g' 's' 'p']


Identificamos que 

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

# define the data preparation for the columns
t = [('cat', OneHotEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
#t = [('cat', OneHotEncoder(drop='first'), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)